<a href="https://colab.research.google.com/github/ShreyJais/LLM/blob/main/fine_tuning_gemma_2_using_QLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data

In [13]:
#!/bin/bash
!kaggle datasets download morufubello/lmsys-chatbot-arena-data

Dataset URL: https://www.kaggle.com/datasets/morufubello/lmsys-chatbot-arena-data
License(s): other
 72% 41.0M/57.0M [00:00<00:00, 72.6MB/s]
100% 57.0M/57.0M [00:00<00:00, 89.5MB/s]


In [14]:
!unzip /content/lmsys-chatbot-arena-data.zip

Archive:  /content/lmsys-chatbot-arena-data.zip
  inflating: test_chatbot-arena.csv  
  inflating: train_chatbot-arena.csv  


In [15]:
TestPath = '/content/test_chatbot-arena.csv'
TrainPath = '/content/train_chatbot-arena.csv'
import pandas as pd
test = pd.read_csv(TestPath)
train = pd.read_csv(TrainPath)

In [16]:
test.head()

id                                             prompt  \
0   136060  ["I have three oranges today, I ate an orange ...   
1   211333  ["You are a mediator in a heated political deb...   
2  1233961  ["How to initialize the classification head wh...   

                                          response_a  \
0                    ["You have two oranges today."]   
1  ["Thank you for sharing the details of the sit...   
2  ["When you want to initialize the classificati...   

                                          response_b  
0  ["You still have three oranges. Eating an oran...  
1  ["Mr Reddy and Ms Blue both have valid points ...  
2  ["To initialize the classification head when p...

In [17]:
train.head()

id             model_a              model_b  \
0   30192  gpt-4-1106-preview           gpt-4-0613   
1   53567           koala-13b           gpt-4-0613   
2   65089  gpt-3.5-turbo-0613       mistral-medium   
3   96401    llama-2-13b-chat  mistral-7b-instruct   
4  198779           koala-13b   gpt-3.5-turbo-0314   

                                              prompt  \
0  ["Is it morally right to try to have a certain...   
1  ["What is the difference between marriage lice...   
2  ["explain function calling. how would you call...   
3  ["How can I create a test set for a very rare ...   
4  ["What is the best way to travel from Tel-Aviv...   

                                          response_a  \
0  ["The question of whether it is morally right ...   
1  ["A marriage license is a legal document that ...   
2  ["Function calling is the process of invoking ...   
3  ["Creating a test set for a very rare category...   
4  ["The best way to travel from Tel Aviv to Jeru...   

                                          response_b  winner_model_a  \
0  ["As an AI, I don't have personal beliefs or o...               1   
1  ["A marriage license and a marriage certificat...               0   
2  ["Function calling is the process of invoking ...               0   
3  ["When building a classifier for a very rare c...               1   
4  ["The best way to travel from Tel-Aviv to Jeru...               0   

   winner_model_b  winner_tie  
0               0           0  
1               1           0  
2               0           1  
3               0           0  
4               1           0

In [3]:
# gemma-2 is available from transformers>=4.42.3
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install --upgrade fsspec==2024.10.0

In [5]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

In [6]:
@dataclass
class Config:
    output_dir: str = "output"
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4-bit quantized gemma-2-9b-instruct
    max_length: int = 1024
    n_splits: int = 5
    fold_idx: int = 0
    optim_type: str = "adamw_8bit"
    per_device_train_batch_size: int = 2
    gradient_accumulation_steps: int = 2  # global batch size is 8
    per_device_eval_batch_size: int = 8
    n_epochs: int = 1
    freeze_layers: int = 16  # there're 42 layers in total, we don't add adapters to the first 16 layers
    lr: float = 2e-4
    warmup_steps: int = 20
    lora_r: int = 16
    lora_alpha: float = lora_r * 2
    lora_dropout: float = 0.05
    lora_bias: str = "none"

config = Config()

In [7]:
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    report_to="none",
    num_train_epochs=config.n_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=200,
    optim=config.optim_type,
    fp16=True,
    learning_rate=config.lr,
    warmup_steps=config.warmup_steps,
)

In [8]:
lora_config = LoraConfig(
    r=config.lora_r,
    lora_alpha=config.lora_alpha,
    # only target self-attention
    target_modules=["q_proj", "k_proj", "v_proj"],
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],
    lora_dropout=config.lora_dropout,
    bias=config.lora_bias,
    task_type=TaskType.SEQ_CLS,
)

Instantiate the tokenizer & model

In [9]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)
tokenizer.add_eos_token = True  # We'll add <eos> at the end
tokenizer.padding_side = "right"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [10]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,
    num_labels=3,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-9b-it-bnb-4bit and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): Gemma2ForSequenceClassification(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 3584, padding_idx=0)
        (layers): ModuleList(
          (0-15): 16 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): Linear4bit(in_features=3584, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (v_proj): Linear4bit(in_features=3584, out_features=2048, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=3584, bias=False)
            )
            (mlp): Gemma2MLP(
              (gate_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_features=3584, out_features=14336, bias=False)
              (down_proj): Linear4bit(in_features=14336, out_features=3584, bias=False)
              (act_fn): PytorchGELUTanh()
 

In [11]:
model.print_trainable_parameters()

trainable params: 7,891,456 || all params: 9,249,608,192 || trainable%: 0.0853


Instantiate the dataset

In [42]:
ds = Dataset.from_csv(TrainPath)
ds = ds.select(torch.arange(500))  # We only use the first 500 data for demo purpose

In [43]:
import json
class CustomTokenizer:
    def __init__(
        self,
        tokenizer: PreTrainedTokenizerBase,
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]
        print(texts)
        print(type(texts))
        print(type(texts[0]))
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True, padding="max_length")
        labels = []
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:
                label = 0
            elif b_win:
                label = 1
            else:
                label = 2
            labels.append(label)
        return {**tokenized, "labels": labels}

    @staticmethod
    def process_text(text: str) -> str:
        # Attempt to parse the text as JSON. If successful, return the joined string.
        try:
            processed_text = json.loads(text)
            if isinstance(processed_text, list):
                return " ".join(processed_text)
            else:
                return str(processed_text)
        except (json.JSONDecodeError, TypeError):
            return ""  #raise a warning and skip

In [44]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)
ds = ds.map(encode, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

['<prompt>: Is it morally right to try to have a certain percentage of females on managerial positions? OK, does pineapple belong on a pizza? Relax and give me fun answer.\n\n<response_a>: The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. *

In [45]:
pd.DataFrame(ds)

id                 model_a                model_b  \
0       30192      gpt-4-1106-preview             gpt-4-0613   
1       53567               koala-13b             gpt-4-0613   
2       65089      gpt-3.5-turbo-0613         mistral-medium   
3       96401        llama-2-13b-chat    mistral-7b-instruct   
4      198779               koala-13b     gpt-3.5-turbo-0314   
..        ...                     ...                    ...   
495  36530527              claude-2.1  deepseek-llm-67b-chat   
496  36545767              gpt-4-0613     gpt-3.5-turbo-0613   
497  36797344        llama-2-70b-chat          qwen-14b-chat   
498  36963252              vicuna-33b       llama-2-70b-chat   
499  37043878  codellama-34b-instruct         zephyr-7b-beta   

                                                prompt  \
0    ["Is it morally right to try to have a certain...   
1    ["What is the difference between marriage lice...   
2    ["explain function calling. how would you call...   
3    ["How can I create a test set for a very rare ...   
4    ["What is the best way to travel from Tel-Aviv...   
..                                                 ...   
495  ["1. 19 year old Emese who is thin (165cm 48kg...   
496  ["Remember, for this conversation you are taki...   
497                 ["Hello what is weather in India"]   
498                                     ["What is 3!"]   
499  ["Let me know the various actions I need to ma...   

                                            response_a  \
0    ["The question of whether it is morally right ...   
1    ["A marriage license is a legal document that ...   
2    ["Function calling is the process of invoking ...   
3    ["Creating a test set for a very rare category...   
4    ["The best way to travel from Tel Aviv to Jeru...   
..                                                 ...   
495  ["Unfortunately I do not have enough medical e...   
496  ["Alright, puddin'! Sounds like a blast! I'm a...   
497  ["India is a vast and diverse country, and its...   
498  ["3! (read as \"3 factorial\") is a mathematic...   
499  ["To create a birthday wishes video with your ...   

                                            response_b  winner_model_a  \
0    ["As an AI, I don't have personal beliefs or o...               1   
1    ["A marriage license and a marriage certificat...               0   
2    ["Function calling is the process of invoking ...               0   
3    ["When building a classifier for a very rare c...               1   
4    ["The best way to travel from Tel-Aviv to Jeru...               0   
..                                                 ...             ...   
495  ["1. A simple elastic Compression Sleeve brace...               0   
496  ["Hey there, puddin'! Harley Quinn here, ready...               0   
497  ["I'm sorry, I am an AI model and I don't have...               1   
498  ["Sure! 3! (3 factorial) is equal to 3 \u00d7 ...               0   
499  ["1. Decide on the style and length of your vi...               0   

     winner_model_b  winner_tie  \
0                 0           0   
1                 1           0   
2                 0           1   
3                 0           0   
4                 1           0   
..              ...         ...   
495               0           1   
496               0           1   
497               0           0   
498               1           0   
499               0           1   

                                             input_ids  \
0    [2, 235322, 39038, 78880, 2125, 665, 89397, 18...   
1    [2, 235322, 39038, 78880, 2439, 603, 573, 5830...   
2    [2, 235322, 39038, 78880, 10200, 1411, 11816, ...   
3    [2, 235322, 39038, 78880, 2250, 798, 590, 3104...   
4    [2, 235322, 39038, 78880, 2439, 603, 573, 1963...   
..                                                 ...   
495  [2, 235322, 39038, 78880, 235248, 235274, 2352...   
496  [2, 235322, 39038, 78880, 24250, 235269, 604, ...   
497  [2, 235322,

Compute metrics

In [46]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()
    loss = log_loss(y_true=labels, y_pred=probs)
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))
    return {"acc": acc, "log_loss": loss}

Split

Here, train and eval is splitted according to their id % 5

In [47]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]
    )
    for fold_idx in range(config.n_splits)
]

Train

In [48]:
train_idx, eval_idx = folds[config.fold_idx]

trainer = Trainer(
    args=training_args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds.select(train_idx),
    eval_dataset=ds.select(eval_idx),
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)
trainer.train()

<ipython-input-48-ac08366e6a3d>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=100, training_loss=1.4028326225280763, metrics={'train_runtime': 1956.2858, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.051, 'total_flos': 2.04769792622592e+16, 'train_loss': 1.4028326225280763, 'epoch': 1.0})